# Analyzing Monsters

## Intro

Summoner's War is a game for mobile platforms where you have monsters... and you summon them. That's about all I know about the game. In this document, I'll be analyzing the statistics of the monsters of Summoner's War and drawing conclusions. I'll be making a lot of assumptions about how to interpret the stats, so this should be fun. All of my conclusions do not take into account any other mechanics like monster abilities, traits, combat rules, monster utility, popularity of other monsters, rarity of monsters, etc. I know so little that I'm not even sure those things exist! So the conclusions I draw here will be drawn purely from stats and my understanding of them, and I'll try to pick the perfect monster for you.

# Setting Up

In [33]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# read monster stats into pandas
df = pd.read_csv('monster_stats.csv')

## Getting Oriented

In [35]:
# first 5 monsters
df.head()

,Name,Element,Grade,HP,ATK,DEF,SPD,Weight
0,Lamor,Fire,2,7245,538,461,100,1482
1,Tigresse,Water,2,6750,582,450,100,1482
2,Samour,Wind,2,7410,582,406,100,1482
3,Varis,Light,2,6420,659,395,100,1482
4,Havana,Dark,2,7080,527,483,100,1482


In [36]:
# last 5 monsters
df.tail()

,Name,Element,Grade,HP,ATK,DEF,SPD,Weight
440,Sekhmet,Fire,6,11205,714,681,99,2142
441,Bastet,Water,6,11850,637,714,99,2141
442,Hathor,Wind,6,11040,692,714,99,2142
443,Isis,Light,6,11700,637,725,99,2142
444,Nephthys,Dark,6,11370,725,659,99,2142


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 8 columns):
Name       445 non-null object
Element    445 non-null object
Grade      445 non-null int64
HP         445 non-null int64
ATK        445 non-null int64
DEF        445 non-null int64
SPD        445 non-null int64
Weight     445 non-null int64
dtypes: int64(6), object(2)
memory usage: 27.9+ KB


In [38]:
df['Element'].value_counts()

Wind     89
Water    89
Dark     89
Light    89
Fire     89
Name: Element, dtype: int64

In [39]:
# formatting functions
format_function = lambda x: "{:>6.2f}".format(x)
right_justify = lambda df: df.style.\
    set_properties(**{'text-align': 'right'})

In [40]:
right_justify(df.describe().applymap(format_function))

Ok, so it looks like there are 445 monsters in Summoner's War. 

"Grade" seems to be some measure of rarity or general strength, because as "grade" increases, the stats increase across the board. "Hit Points" and "Defense" sound like defensive stats, so I think they determine how much of a beating your monster can take, where "hit points" is how many points of health available and "defense" is damage reduction. "Attack" seems like an offensive stat, telling you how hard your monster can hit. "Speed" is a toss-up. Perhaps the more "speed" your monster has, the earlier it can attack in a battle. Or perhaps more "speed" means it can maneuver faster, so it's better at dodging attacks. I'm going to assume the former, so I think "speed" is an offensive stat, synergizing with "attack". If you attack fast and hard, you can eliminate threats before they can do anything. As for weight, I'm clueless. Maybe Summoner's War wants to balance how many monsters you can carry at a time, so you have a maximum total weight you can hold. Maybe it goes up as your summoner gets stronger from carrying these heavy monsters all the time. Maybe Summoner's War is actually a sumo wrestling game, where it's a lot easier for heavy monsters to push lighter monsters out of the ring. I don't know.

There are 5 elements: water, light, dark, wind, and fire. Each element has 89 monsters, so they look great for comparing.

Something interesting to note is that "Grade" is a range of 2-6, but its mean is 4.58. With a range of 2-6, I expected the mean to be close to 4, but 4.58 is more than halfway to 5, so this means that there are quite a few more high "grade" monsters than low grade. 

Speed seems to the be the stat that changes the least, with a standard deviation of 6.03, so maybe speed doesn't play much of a factor most of the time. But when there is a speed difference, there surely must be large implications. 

## Elemental Groups

Next, I'll analyze the monsters when they are split into their element groups.

In [41]:
# get all of the elements in the dataframe
elements = set(df['Element'])
elements

{'Dark', 'Fire', 'Light', 'Water', 'Wind'}

In [42]:
# create a dictionary that maps from element names to dataframes for the monsters of those elements
monsters_by_element = {element: df[df['Element'] == element] for element in elements}

In [43]:
monsters_by_element['Fire'].describe().applymap(format_function)

,Grade,HP,ATK,DEF,SPD,Weight
count,89.00,89.00,89.00,89.00,89.00,89.00
mean,4.56,9707.70,690.64,566.54,102.51,1904.36
std,0.94,1532.69,123.36,88.40,5.83,155.11
min,2.00,5595.00,384.00,340.00,89.00,1482.00
25%,4.00,8565.00,604.00,505.00,99.00,1812.00
50%,5.00,9720.00,703.00,571.00,102.00,1976.00
75%,5.00,10875.00,780.00,637.00,105.00,1977.00
max,6.00,13005.00,911.00,790.00,126.00,2142.00


In [44]:
monsters_by_element['Water'].describe().applymap(format_function)

,Grade,HP,ATK,DEF,SPD,Weight
count,89.00,89.00,89.00,89.00,89.00,89.00
mean,4.54,9720.34,671.64,581.00,102.21,1900.66
std,0.97,1607.89,138.09,85.83,5.97,159.17
min,2.00,5265.00,351.00,362.00,87.00,1482.00
25%,4.00,8565.00,582.00,527.00,99.00,1812.00
50%,5.00,9885.00,714.00,582.00,102.00,1976.00
75%,5.00,10710.00,769.00,659.00,105.00,1977.00
max,6.00,13170.00,988.00,801.00,119.00,2142.00


In [45]:
# find the fire monster with the most hp
max_hp = monsters_by_element['Fire']['HP'].max()
monsters_by_element['Fire'][monsters_by_element['Fire']['HP'] == max_hp]

,Name,Element,Grade,HP,ATK,DEF,SPD,Weight
406,Kumar,Fire,6,13005,593,681,96,2141


In [46]:
# find the fire monster with the most attack
max_atk = monsters_by_element['Fire']['ATK'].max()
monsters_by_element['Fire'][monsters_by_element['Fire']['ATK'] == max_atk]

,Name,Element,Grade,HP,ATK,DEF,SPD,Weight
85,Tagaros,Fire,4,7740,911,384,95,1811
361,Zaiross,Fire,6,9720,911,582,94,2141


In [47]:
# describe the stats of the hp column for the fire monsters
monsters_by_element['Fire']['HP'].describe()

count       89.000000
mean      9707.696629
std       1532.692420
min       5595.000000
25%       8565.000000
50%       9720.000000
75%      10875.000000
max      13005.000000
Name: HP, dtype: float64

In [48]:
# print the max health for each element
for element in elements:
    print('{:5}: {}'.format(element, monsters_by_element[element]['HP'].max()))

Water: 13170
Wind : 13500
Fire : 13005
Light: 15315
Dark : 12675


In [49]:
def max_attributes(stat):
    '''Create a dictionary of Series which contain the max stats for each element'''
    result = dict()
    for element in elements:
        result[element] = monsters_by_element[element][stat].max()
    result = pd.Series(result)
    result.name = stat
    return result

In [50]:
hp = max_attributes('HP')
hp

Dark     12675
Fire     13005
Light    15315
Water    13170
Wind     13500
Name: HP, dtype: int64

In [51]:
atk = max_attributes('ATK')
atk

Dark     1021
Fire      911
Light    1054
Water     988
Wind     1098
Name: ATK, dtype: int64

In [52]:
# combine the Series-es into a dataframe
pd.DataFrame([hp, atk]).transpose()

,HP,ATK
Dark,12675,1021
Fire,13005,911
Light,15315,1054
Water,13170,988
Wind,13500,1098


In [53]:
# create a dataframe for all the max stats except for Grade
#     because I don't care what the max grade is. I know it's 6
max_stats = pd.DataFrame([max_attributes(stat) for stat \
                          in ['HP', 'ATK', 'SPD', 'DEF', 'Weight']]).transpose()

In [54]:
# taken from pandas documentation
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [55]:
def highlight_min(data, color='lightblue'):
    '''
    highlight the minimum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_min = data == data.min()
        return [attr if v else '' for v in is_min]
    else:  # from .apply(axis=None)
        is_min = data == data.min().min()
        return pd.DataFrame(np.where(is_min, attr, ''),
                            index=data.index, columns=data.columns)

In [56]:
# print the table and highlight the max stats in each column
max_stats.style.apply(highlight_max)

Each element has a monster with a weight of 2142, so nothing special there. Maybe each element gets a legendary monster that weighs that much.

Light holds the monster with the highest HP by far. Wind is home to the monster with the highest attack. Fire contains the speediest monster. Dark houses the most defensive monster. Water seems to be left in the dust with none of its monsters breaking any records.

In [57]:
def mean_attributes(stat):
    '''Create a dictionary of Series which contain the mean stats for each element'''
    result = dict()
    for element in elements:
        result[element] = monsters_by_element[element][stat].mean()
    result = pd.Series(result)
    result.name = stat
    return result

In [58]:
# collect the mean stats for each element
mean_stats = pd.DataFrame([mean_attributes(stat) \
                           for stat in ['HP', 'ATK', 'SPD', 'DEF', 'Weight']]).transpose()

In [59]:
# highlight the maxs in yellow and the mins in light blue
mean_stats.style.apply(highlight_max).apply(highlight_min)

In [60]:
mean_stats['HP'] - mean_stats['HP'].min()

Dark     177.977528
Fire     109.213483
Light    531.404494
Water    121.853933
Wind       0.000000
Name: HP, dtype: float64

Next, I'm going to try to find the *relative* stats, so I'll be comparing each column its minimum value. This is useful for when we want to see how each element compares to each other in stats.

In [61]:
relative_mean_stats = pd.DataFrame([mean_stats[stat] - mean_stats[stat].min() \
                                    for stat in mean_stats.columns]).transpose()

In [62]:
relative_mean_stats

,HP,ATK,SPD,DEF,Weight
Dark,177.977528,37.044944,0.528090,8.022472,22.191011
Fire,109.213483,31.910112,0.292135,2.955056,7.404494
Light,531.404494,0.000000,0.449438,16.056180,16.741573
Water,121.853933,12.910112,0.000000,17.415730,3.707865
Wind,0.000000,34.741573,1.202247,0.000000,0.000000


In [63]:
relative_mean_stats.applymap(format_function)\
    .style.apply(highlight_max).apply(highlight_min)\
    .set_properties(**{'text-align': 'right'})

Wow, there's a lot we can find here. 

On average, **Dark** monsters have high attack and significantly higher weight than the other classes. Speed, HP, and defense are middle-of-the-road, so Dark seems to be the all-rounder elemental class.

**Fire** monsters are also middle-of-the-road monsters, except they are outclassed in every way by dark monsters on average. Sucks to be a fire monster.

**Light** monsters have *significantly* higher HP than the other classes. The other classes are clustered around 130, with one at 0, and Light is 353 away from its nearest contender. That's almost triple the other classes' distance from the bottom! On top of that, Light has the second-best defense stat and second-best weight. It also has the worst attack stat and mediocre speed. So Light monsters are about tanking hits. If you want to stall out a game, maybe set up some other monsters, you want a Light monster on your front line. Sure, if you ask a Light monster to attack your enemies, it'll be the equivalent of them trying to swing a wet noodle at them, but if you can find a Light monster that does anything else but attack, you have yourself a real winner. As a protection fighter or a supporting tank, the light monster could buff allies, debuff enemies, or boost its own defense, all while soaking a ton of damage.

**Water** monsters have mediocre HP and the highest defense. They weigh very little, and they're the weakest in speed. They are about halfway between Light and the other classes in attack, though. I think you would want to pick a Water monster when damage mitigation is important (Defense), you know the enemies have no way of ignoring that mitigation, and you want a monster that can deal some damage too. Perhaps you'd want a Light monster in the very front and then a Water monster backing him up, dishing out some damage and taking some too. Water seems to be, on average, a bruiser class.

**Wind** monsters lose out in 3 categories: HP, Defense, and Weight. They also have the highest speed by far, given that speed differences seem to matter a lot (see *Getting Oriented*). They command the second-highest attack, not far from the #1 spot (Dark). What Dark lacks in speed, Wind more than makes up for it. On average, wind is extremely weak in defensive stats and top-tier in offensive stats, so I would label Wind the "glass cannon" class. You want these monsters in the back line, darting in and out of combat, lobbing powerful attacks, eliminating threats before they can pop off. If the enemy even shoots them a dirty look, they'll explode, so you want to make sure they take as little damage as possible. 

In [64]:
# todo: recommend some specific monsters. 
#       like a DREAM TEAM of monsters to find and train.